<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h1>Case Challenge Part II</h1>
<b>DAT-5303 | Machine Learning</b><br>
Arline Garin<br>
Hult International Business School<br><br><br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br>

In [1]:
# importing libraries
import random            as rand                     # random number gen
import pandas            as pd                       # data science essentials
import matplotlib.pyplot as plt                      # data visualization
import seaborn           as sns                      # enhanced data viz
import numpy             as np 
from sklearn.model_selection import train_test_split # train-test split
from sklearn.linear_model import LogisticRegression  # logistic regression
import statsmodels.formula.api as smf                # logistic regression
from sklearn.metrics import confusion_matrix         # confusion matrix
from sklearn.metrics import roc_auc_score            # auc score
from sklearn.neighbors import KNeighborsClassifier   # KNN for classification
from sklearn.neighbors import KNeighborsRegressor    # KNN for regression
from sklearn.preprocessing import StandardScaler     # standard scaler
from sklearn.tree import DecisionTreeClassifier      # classification trees
from sklearn.tree import export_graphviz             # exports graphics
from six import StringIO           # saves objects in memory
from IPython.display import Image                    # displays on frontend
import pydotplus                                     # interprets dot objects
from sklearn.model_selection import RandomizedSearchCV     # hyperparameter tuning
from sklearn.metrics import make_scorer              # customizable scorer
from sklearn.ensemble import RandomForestClassifier     # random forest
from sklearn.ensemble import GradientBoostingClassifier # gbm

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# specifying the path and file name
file = './Apprentice_Chef_Dataset.xlsx'


# reading the file into Python
chef = pd.read_excel(io = file)

In [2]:
#########################
# mv_flagger
#########################
def mv_flagger(df):
    """
Flags all columns that have missing values with 'm-COLUMN_NAME'.

PARAMETERS
----------
df : DataFrame to flag missing values


RETURNS
-------
DataFrame with missing value flags."""


    for col in df:

        if df[col].isnull().astype(int).sum() > 0:
            df['m_'+col] = df[col].isnull().astype(int)
            
    return df

In [3]:
#########################
# text_split_feature
#########################
def text_split_feature(col, df, sep=' ', new_col_name='number_of_names'):
    """
Splits values in a string Series (as part of a DataFrame) and sums the number
of resulting items. Automatically appends summed column to original DataFrame.

PARAMETERS
----------
col          : column to split
df           : DataFrame where column is located
sep          : string sequence to split by, default ' '
new_col_name : name of new column after summing split, default
               'number_of_names'
"""
    
    df[new_col_name] = 0
    
    
    for index, val in df.iterrows():
        df.loc[index, new_col_name] = len(df.loc[index, col].split(sep = ' '))

In [4]:
########################################
# visual_cm
########################################
def visual_cm(true_y, pred_y, labels = None):
    """
Creates a visualization of a confusion matrix.

PARAMETERS
----------
true_y : true values for the response variable
pred_y : predicted values for the response variable
labels : , default None
    """
    # visualizing the confusion matrix

    # setting labels
    lbls = labels
    

    # declaring a confusion matrix object
    cm = confusion_matrix(y_true = true_y,
                          y_pred = pred_y)


    # heatmap
    sns.heatmap(cm,
                annot       = True,
                xticklabels = lbls,
                yticklabels = lbls,
                cmap        = 'Blues',
                fmt         = 'g')


    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix of the Classifier')
    plt.show()

In [5]:
########################################
# plot_feature_importances
########################################
def plot_feature_importances(model, train, export = False):
    """
    Plots the importance of features from a CART model.
    
    PARAMETERS
    ----------
    model  : CART model
    train  : explanatory variable training data
    export : whether or not to export as a .png image, default False
    """
    
    # declaring the number
    n_features = x_train.shape[1]
    
    # setting plot window
    fig, ax = plt.subplots(figsize=(12,9))
    
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(pd.np.arange(n_features), train.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    
    if export == True:
        plt.savefig('Tree_Leaf_50_Feature_Importance.png')

In [6]:
########################################
# display_tree
########################################
def display_tree(tree, feature_df, height = 500, width = 800, export = False):
    """
    PARAMETERS
    ----------
    tree       : fitted tree model object
        fitted CART model to visualized
    feature_df : DataFrame
        DataFrame of explanatory features (used to generate labels)
    height     : int, default 500
        height in pixels to which to constrain image in html
    width      : int, default 800
        width in pixels to which to constrain image in html
    export     : bool, defalut False
        whether or not to export the tree as a .png file
    """

    # visualizing the tree
    dot_data = StringIO()

    
    # exporting tree to graphviz
    export_graphviz(decision_tree      = tree,
                    out_file           = dot_data,
                    filled             = True,
                    rounded            = True,
                    special_characters = True,
                    feature_names      = feature_df.columns)


    # declaring a graph object
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())


    # creating image
    img = Image(graph.create_png(),
                height = height,
                width  = width,
                unconfined = True)


    return img

## Exploratory Data Analysis - EDA

In [7]:
# Renaming the mislabeled column 
chef.rename(columns = {'LARGEST_ORDER_SIZE':'AVG_NB_MEALS'}, inplace = True)

### Analyzing the Y-variable

In [8]:
# Differentiating success and failure of the promotion

SUCCEEDED_PROMOTION   = len(chef['CROSS_SELL_SUCCESS'][chef['CROSS_SELL_SUCCESS'] == 1])
FAILED_PROMOTION      = len(chef['CROSS_SELL_SUCCESS'][chef['CROSS_SELL_SUCCESS'] == 0])

print(f"""
                                    ---------------------
Suceeded Cross-Sell Promotion       | {SUCCEEDED_PROMOTION}
Failed Cross-Sell Promotion         | {FAILED_PROMOTION}
""")



                                    ---------------------
Suceeded Cross-Sell Promotion       | 1321
Failed Cross-Sell Promotion         | 625



In [9]:
# creating a (Pearson) correlation matrix
df_corr = chef.corr().round(2)


# printing (Pearson) correlations with SalePrice
print(df_corr.loc['CROSS_SELL_SUCCESS'].sort_values(ascending = False))

CROSS_SELL_SUCCESS             1.00
CANCELLATIONS_BEFORE_NOON      0.16
MOBILE_NUMBER                  0.10
TASTES_AND_PREFERENCES         0.08
REFRIGERATED_LOCKER            0.07
PC_LOGINS                      0.04
MASTER_CLASSES_ATTENDED        0.04
PACKAGE_LOCKER                 0.04
CONTACTS_W_CUSTOMER_SERVICE    0.04
MEDIAN_MEAL_RATING             0.03
AVG_PREP_VID_TIME              0.03
AVG_NB_MEALS                   0.02
EARLY_DELIVERIES               0.02
TOTAL_MEALS_ORDERED            0.01
AVG_TIME_PER_SITE_VISIT        0.01
TOTAL_PHOTOS_VIEWED            0.01
LATE_DELIVERIES                0.01
PRODUCT_CATEGORIES_VIEWED      0.00
UNIQUE_MEALS_PURCH             0.00
REVENUE                        0.00
WEEKLY_PLAN                   -0.01
AVG_CLICKS_PER_VISIT          -0.04
CANCELLATIONS_AFTER_NOON      -0.05
MOBILE_LOGINS                 -0.05
Name: CROSS_SELL_SUCCESS, dtype: float64


### Review of Missing Value Analysis and Imputation

In [10]:
# Missing Value Flagger uses in place in the function            
chef = mv_flagger(chef)

# Imputing NA's
chef.loc[:,'FAMILY_NAME'].fillna('Unknown',inplace = True)


# chef.columns # Checking which columns where created.

# chef.isnull().any().any() # making sure all missing values have been taken care of

### Data Measurement Classification

<b>Heuristics of classification:</b>

- Continuous: Numerical values
- Binary: Containing 1 and 0
- Count or interval: How many of something was measured.
- Discrete / Nominal: Unique variables.


<b>Continuous</b>
- REVENUE
- AVG_TIME_PER_SITE_VISIT
- AVG_PREP_VID_TIME
- AVG_CLICKS_PER_VISIT

<b>Binary</b>
- CROSS-SELL SUCCESS
- MOBILE_NUMBER
- TASTES_AND_PREFERENCES
- PACKAGE_LOCKER
- REFRIGERATED_LOCKER

<b>Count or Interval</b>
- TOTAL_MEALS_ORDERED
- UNIQUE_MEALS_PURCH
- CONTACTS_W_CUSTOMER_SERVICE
- PRODUCT_CATEGORIES_VIEWED
- CANCELLATIONS_BEFORE_NOON
- CANCELLATIONS_AFTER_NOON
- PC_LOGINS
- MOBILE_LOGINS
- WEEKLY_PLAN
- EARLY_DELIVERIES
- LATE_DELIVERIES
- AVG_NB_MEALS
- MASTER_CLASSES_ATTENDED
- TOTAL_PHOTOS_VIEWED
- MEDIAN_MEAL_RATING

<b>Discrete or Nominal</b>
- NAME
- EMAIL
- FIRST_NAME
- LAST_NAME

In [11]:
# chef.WEEKLY_PLAN.describe() # Count of weeks that a customer subscribed 

# It seems that users are not using the weekly meal plans! 
# chef[chef.WEEKLY_PLAN == 52]
# chef.TOTAL_MEALS_ORDERED.describe()

chef.loc[chef.TOTAL_MEALS_ORDERED > 95, ['WEEKLY_PLAN',
                                      'NAME',
                                      'TOTAL_MEALS_ORDERED',
                                     'CROSS_SELL_SUCCESS']].sort_values('WEEKLY_PLAN',
                                                ascending = False).groupby('CROSS_SELL_SUCCESS').mean()
# It looks that there's no effective cross sell success for people that order a lot of meals

chef['HIGH_TOTAL_MEALS_ORDERED'] = 0
chef.loc[chef.TOTAL_MEALS_ORDERED > 95, 'HIGH_TOTAL_MEALS_ORDERED'] = 1

In [12]:
chef.loc[chef.TOTAL_MEALS_ORDERED < 60, ['WEEKLY_PLAN',
                                      'NAME',
                                      'TOTAL_MEALS_ORDERED',
                                     'CROSS_SELL_SUCCESS']].sort_values('WEEKLY_PLAN',
                                            ascending = False).groupby('CROSS_SELL_SUCCESS').mean()
# It looks that the Total Meals people that has ordered less than 60 meals are 
# slightly more prone to do succesfull cross sell 

chef['LOW_TOTAL_MEALS_ORDERED'] = 0
chef.loc[chef.TOTAL_MEALS_ORDERED < 60, 'LOW_TOTAL_MEALS_ORDERED'] = 1

In [13]:
# New dummy variable for customers using the mobile app
chef['USE_MOBILE_APP'] = 0
chef.loc[chef.MOBILE_LOGINS > 0, 'USE_MOBILE_APP'] = 1

## Feature Engineering

### Gender Guesser

In [14]:
# import gender_guesser.detector as gender # guess gender based on (given) name

# guessing gender based on (given) name

# placeholder list
#placeholder_lst = []


# looping to guess gender
#for name in chef ['FIRST_NAME']:
    #guess = gender.Detector().get_gender(name)
    #print(guess)
    #placeholder_lst.append(guess)
    
# checking results
#print(placeholder_lst)

# Adding the guessed gender from gender_guesser.detector into a new column the dataframe 
chef['GENDER_GUESS'] = ['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'mostly_male', 'female', 'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'male', 'male', 'male', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'male', 'unknown', 'female', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'female', 'female', 'unknown', 'male', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'andy', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'mostly_male', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_male', 'unknown', 'mostly_male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'mostly_male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'mostly_male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 'andy', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'andy', 'male', 'unknown', 'unknown', 'male', 'male', 'female', 'female', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'female', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'mostly_female', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'female', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'mostly_female', 'unknown', 'male', 'unknown', 'female', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'male', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'female', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'male', 'male', 'mostly_male', 'male', 'male', 'male', 'male', 'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'andy', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'male', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'female', 'male', 'male', 'female', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_male', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'andy', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'unknown', 'female', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'andy', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'mostly_male', 'male', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'mostly_male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'male', 'unknown', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'female', 'male', 'male', 'unknown', 'male', 'unknown', 'mostly_female', 'male', 'unknown', 'unknown', 'female', 'male', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'female', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'female', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'mostly_male', 'mostly_male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'female', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'male', 'male', 'mostly_male', 'unknown', 'unknown', 'male', 'andy', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'female', 'male', 'female', 'mostly_female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'mostly_female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'female', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'male', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'mostly_female', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'mostly_female', 'female', 'female', 'male', 'male', 'male', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'female', 'unknown', 'male', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'unknown', 'andy', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'mostly_male', 'male', 'male', 'unknown', 'male', 'unknown', 'mostly_male', 'female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'unknown', 'mostly_male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'male', 'unknown', 'female', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'mostly_female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'andy', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'mostly_male', 'unknown', 'male', 'male', 'unknown', 'unknown', 'male', 'male', 'male', 'male', 'andy', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'female', 'unknown', 'unknown']

# Checking results
# chef['GENDER_GUESS'].value_counts()

## OUTPUT
# unknown          1385
# male              381
# female            125
# mostly_male        24
# mostly_female      21
# andy               10
# Name: GENDER_GUESS, dtype: int64

# Reducing the number of "unknown" gender 

for index, row in chef.iterrows():

    # First nested condition for Gender guessed as "unknown"
    if 'unknown' in chef.loc[index, 'GENDER_GUESS']:

        if chef.loc[index, 'FIRST_NAME'].endswith('a'):
            chef.loc[index, 'GENDER_GUESS'] = 'female'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('o'):
            chef.loc[index, 'GENDER_GUESS'] = 'male'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('r'):
            chef.loc[index, 'GENDER_GUESS'] = 'male'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('d'):
            chef.loc[index, 'GENDER_GUESS'] = 'male'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('lin'):
            chef.loc[index, 'GENDER_GUESS'] = 'female'

        elif chef.loc[index, 'FIRST_NAME'].endswith('lyn'):
            chef.loc[index, 'GENDER_GUESS'] = 'female'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('on'):
            chef.loc[index, 'GENDER_GUESS'] = 'male'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('os'):
            chef.loc[index, 'GENDER_GUESS'] = 'male'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('ne'):
            chef.loc[index, 'GENDER_GUESS'] = 'female'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('en'):
            chef.loc[index, 'GENDER_GUESS'] = 'male'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('s'):
            chef.loc[index, 'GENDER_GUESS'] = 'male'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('se'):
            chef.loc[index, 'GENDER_GUESS'] = 'female'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('k'):
            chef.loc[index, 'GENDER_GUESS'] = 'male'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('e'):
            chef.loc[index, 'GENDER_GUESS'] = 'female'
            
        elif chef.loc[index, 'FIRST_NAME'].endswith('y'):
            chef.loc[index, 'GENDER_GUESS'] = 'female'
    
    # Second condition for Gender guessed as "mostly_male"
    elif 'mostly_male' in chef.loc[index, 'GENDER_GUESS']:
        chef.loc[index, 'GENDER_GUESS'] = 'male'
    
    # Third condition for Gender guessed as "mostly_female"
    elif 'mostly_female' in chef.loc[index, 'GENDER_GUESS']:
        chef.loc[index, 'GENDER_GUESS'] = 'female'
    
    # Fourth condition for Gender guessed as "andy"
    elif 'andy' in chef.loc[index, 'GENDER_GUESS']:
        chef.loc[index, 'GENDER_GUESS'] = 'male'
        
# Checking results
# chef['GENDER_GUESS'].value_counts()


## OUTPUT
# male       1082
# female      516
# unknown     348
# Name: GENDER_GUESS, dtype: int64

# categorical_boxplots(response = 'LOG_REVENUE', cat_var = 'GENDER_GUESS', data = chef)

# one hot encoding GENDER categorical variable
GENDER_DUMMIES       = pd.get_dummies(chef['GENDER_GUESS'])

# dropping categorical variables after they've been encoded
chef = chef.drop('GENDER_GUESS', axis = 1)

# joining codings together
chef = chef.join([GENDER_DUMMIES])

# saving new columns
new_columns = chef.columns

### Email Adresses

In [15]:
# Divide the emails by domain knowledge provided by Customer Service Team
professional = ["mmm.com", "amex.com", "apple.com", "boeing.com", "caterpillar.com",
    "chevron.com", "cisco.com", "cocacola.com", "disney.com", "dupont.com",
    "exxon.com", "ge.org", "goldmansacs.com", "homedepot.com", "ibm.com",
    "intel.com", "jnj.com", "jpmorgan.com", "mcdonalds.com", "merck.com",
    "microsoft.com", "nike.com", "pfizer.com", "pg.com", "travelers.com",
    "unitedtech.com", "unitedhealth.com", "verizon.com", "visa.com",
    "walmart.com"]

personal = ["gmail.com", "yahoo.com", "protonmail.com"]

junk = ["me.com", "aol.com", "hotmail.com", "live.com", "msn.com", "passport.com"]

# Split by @ and select domains
chef['EMAIL_DOMAIN'] = chef.EMAIL.str.split('@', expand=True)[1]

# Create email_type column
chef['EMAIL_TYPE'] = '0'

# Populate the Email Type column by if else statements
for index, row in chef.iterrows():

    if chef.loc[index, 'EMAIL_DOMAIN'] in professional:
        chef.loc[index, 'EMAIL_TYPE'] = 'professional'
    elif chef.loc[index, 'EMAIL_DOMAIN'] in personal:
        chef.loc[index, 'EMAIL_TYPE'] = 'personal'
    elif chef.loc[index, 'EMAIL_DOMAIN'] in junk:
        chef.loc[index, 'EMAIL_TYPE'] = 'junk'
    elif chef.loc[index, 'EMAIL_DOMAIN'] not in professional or chef.loc[row, 'EMAIL'] not in personal or chef.loc[row, 'EMAIL'] not in junk:
        chef.loc[index, 'EMAIL_TYPE'] = 'unknown'
    else:
        print('Houston, we have a problem')

# Checking results
chef['EMAIL_TYPE'].value_counts()

## OUTPUT

# personal        861
# professional    696
# junk            389
# Name: EMAIL_TYPE, dtype: int64

# Distribution of Email Type
# categorical_boxplots(response = 'LOG_REVENUE', cat_var = 'EMAIL_TYPE', data = chef)

# one hot encoding categorical variable
EMAIL_TYPE_DUMMIES       = pd.get_dummies(chef['EMAIL_TYPE'])

# dropping categorical variables after they've been encoded
chef = chef.drop('EMAIL_TYPE', axis = 1)

# joining codings together
chef = chef.join([EMAIL_TYPE_DUMMIES])

# saving new columns
new_columns = chef.columns

### Names

In [16]:
# New variable counting the number of names in NAME column        
text_split_feature(col = 'NAME', df = chef, new_col_name = 'NUMBER_NAMES' )

In [17]:
# New variable to flag when FIRST NAME == FAMILY NAME
placeholder_lst = []

for row,col in chef.iterrows():
    if chef.loc[row,'FIRST_NAME'] == chef.loc[row,'FAMILY_NAME']:
        placeholder_lst.append(1)
    else:
        placeholder_lst.append(0)

# Adding the new variable to the dataset
chef['SAME_NAME'] = pd.Series(placeholder_lst)

In [18]:
# New variable to flag NOBLE customers
placeholder_lst = []

for row,pattern in chef.iterrows():
    if ' of ' in chef.loc[row,'NAME'] or \
    'lord' in chef.loc[row,'NAME'] or \
    'Lord' in chef.loc[row,'NAME'] or \
    ' mo ' in chef.loc[row,'NAME'] or \
    ' zo ' in chef.loc[row,'NAME'] or \
    ' Mo ' in chef.loc[row,'NAME'] or \
    'Knight' in chef.loc[row, 'NAME'] or \
    'knight'in chef.loc[row, 'NAME']:
        placeholder_lst.append(1)
    else:
        placeholder_lst.append(0)

chef['NOBLE'] = pd.Series(placeholder_lst)

### Master Classes

In [19]:
# Dummy variables for customers who have attended master classes
chef['HAS_ATTENDED_MASTER_CLASS']   = 0

# Iterating over original columns to change values in the new feature columns
for index, value in chef.iterrows():
    
    # Has Attended Master Classes     
    if chef.loc[index, 'MASTER_CLASSES_ATTENDED'] > 0:
        chef.loc[index, 'HAS_ATTENDED_MASTER_CLASS'] = 1

In [20]:
# declaring explanatory variables
chef_data = chef.drop(['CROSS_SELL_SUCCESS', 'NAME',
                      'EMAIL', 'FIRST_NAME', 'FAMILY_NAME', 'EMAIL_DOMAIN','m_FAMILY_NAME', 'unknown', 
                       'personal', 'TOTAL_MEALS_ORDERED'], axis = 1)


# declaring response variable
chef_target = chef.loc [ : , 'CROSS_SELL_SUCCESS']

In [21]:
# train-test split with stratification
x_train, x_test, y_train, y_test = train_test_split(
            chef_data,
            chef_target,
            test_size    = 0.25,
            random_state = 219,
            stratify     = chef_target)


# merging training data for statsmodels
chef_train = pd.concat([x_train, y_train], axis = 1)

In [22]:
# instantiating a logistic regression model object
#logit_sig = smf.logit(formula = """ CROSS_SELL_SUCCESS ~ 
#                                     MOBILE_NUMBER + 
#                                     CANCELLATIONS_BEFORE_NOON +
#                                     TASTES_AND_PREFERENCES +  
#                                     MOBILE_LOGINS +
#                                     EARLY_DELIVERIES +
#                                     REFRIGERATED_LOCKER +
#                                     USE_MOBILE_APP + 
#                                     female +
#                                     junk +  
#                                     professional +
#                                     SAME_NAME +
#                                     HAS_ATTENDED_MASTER_CLASS""",
#                                            data    = chef_train)


# fitting the model object
#logit_sig = logit_sig.fit()


# checking the results SUMMARY
#logit_sig.summary()

## Creation of dictionary & Train Tests

In [23]:
dict1 = {

 # full model
 'logit_full'   : ['REVENUE', 'TOTAL_MEALS_ORDERED', 'UNIQUE_MEALS_PURCH', 'CONTACTS_W_CUSTOMER_SERVICE', 
                   'PRODUCT_CATEGORIES_VIEWED', 'AVG_TIME_PER_SITE_VISIT', 'MOBILE_NUMBER', 
                   'CANCELLATIONS_BEFORE_NOON', 'CANCELLATIONS_AFTER_NOON', 'TASTES_AND_PREFERENCES', 
                   'PC_LOGINS', 'MOBILE_LOGINS', 'WEEKLY_PLAN', 'EARLY_DELIVERIES', 'LATE_DELIVERIES', 
                   'PACKAGE_LOCKER', 'REFRIGERATED_LOCKER', 'AVG_PREP_VID_TIME', 'AVG_NB_MEALS', 
                   'MASTER_CLASSES_ATTENDED', 'MEDIAN_MEAL_RATING', 'AVG_CLICKS_PER_VISIT', 
                   'TOTAL_PHOTOS_VIEWED','USE_MOBILE_APP', 'female', 'male', 'unknown', 
                   'junk', 'personal', 'NUMBER_NAMES', 'SAME_NAME', 'NOBLE', 'HAS_ATTENDED_MASTER_CLASS'],
 

 # significant variables only (set 1)
 'logit_sig'    : ['MOBILE_NUMBER', 'CANCELLATIONS_BEFORE_NOON', 'CANCELLATIONS_AFTER_NOON', 
                   'TASTES_AND_PREFERENCES', 'MOBILE_LOGINS','EARLY_DELIVERIES', 
                   'REFRIGERATED_LOCKER', 'AVG_PREP_VID_TIME', 'USE_MOBILE_APP','female', 'junk', 
                   'professional', 'NUMBER_NAMES', 'SAME_NAME', 'NOBLE', 'HAS_ATTENDED_MASTER_CLASS'],

}

In [24]:
# train/test split with the sig model
chef_data   =  chef.loc[ : , dict1['logit_sig']]
chef_target =  chef.loc[ : , 'CROSS_SELL_SUCCESS']


# Train test split with the significant variables
x_train, x_test, y_train, y_test = train_test_split(
            chef_data,
            chef_target,
            test_size    = 0.25,
            random_state = 219,
            stratify     = chef_target)

In [25]:
# train/test split with the full model
chef_data_full   =  chef.loc[ : , dict1['logit_full']]
chef_target_full =  chef.loc[ : , 'CROSS_SELL_SUCCESS']


# Train test split with the full dataset
X_train, X_test, y_train, y_test = train_test_split(
            chef_data_full,
            chef_target_full,
            test_size    = 0.25,
            random_state = 219,
            stratify     = chef_target_full)

In [26]:
# Divide a scaled data set into train and test variable

chef_data_full   =  chef.loc[ : , dict1['logit_full']]
chef_target_full =  chef.loc[ : , 'CROSS_SELL_SUCCESS']

# INSTANTIATING StandardScaler()
scaler = StandardScaler()

# FITTING the independent variable data
scaler.fit(chef_data)


# TRANSFORMING the independent variable data
X_scaled     = scaler.transform(chef_data)


# converting to a DataFrame
X_scaled_chef  = pd.DataFrame(X_scaled) 


# Train test split with all the scaled data
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
                    X_scaled_chef,
                    chef_target_full,
                    random_state = 219,
                    test_size = 0.25,
                    stratify = chef_target_full)

## Logistic Regression

In [27]:
# INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'lbfgs',
                            C = 1,
                            max_iter = 500,
                            random_state = 219)


# FITTING the training data
logreg_fit = logreg.fit(x_train, y_train)


# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(x_test)


# SCORING the results
print('Training Score:', logreg_fit.score(x_train, y_train).round(4))
print('Testing Score:', logreg_fit.score(x_test, y_test).round(4))
print('AUC Score:', roc_auc_score(y_true  = y_test,
                    y_score = logreg_pred).round(decimals = 4))

# saving scoring data for future use
logreg_train_score = logreg_fit.score(x_train, y_train).round(4) # accuracy
logreg_test_score  = logreg_fit.score(x_test, y_test).round(4) # accuracy
logreg_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = logreg_pred).round(decimals = 4) #auc

Training Score: 0.7608
Testing Score: 0.7618
AUC Score: 0.6858


In [28]:
# unpacking the confusion matrix
logreg_tn, \
logreg_fp, \
logreg_fn, \
logreg_tp = confusion_matrix(y_true = y_test, y_pred = logreg_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {logreg_tn}
False Positives: {logreg_fp}
False Negatives: {logreg_fn}
True Positives : {logreg_tp}
""")


True Negatives : 74
False Positives: 82
False Negatives: 34
True Positives : 297



In [29]:
# INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'lbfgs',
                            C = 4,
                            random_state = 219)


# FITTING the training data
logreg_fit = logreg.fit(X_train_scaled, y_train_scaled)


# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(X_test_scaled)


# SCORING the results
print('Training ACCURACY:', logreg_fit.score(X_train_scaled, y_train_scaled).round(4))
print('Testing  ACCURACY:', logreg_fit.score(X_test_scaled, y_test_scaled).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test_scaled,
                                          y_score = logreg_pred).round(4))


Training ACCURACY: 0.7622
Testing  ACCURACY: 0.7618
AUC Score        : 0.6875


### Hyperparameter Tuning With RandomizedSearchCV

In [30]:
########################################
# RandomizedSearchCV
########################################

# declaring a hyperparameter space
C_space          = pd.np.arange(1.0, 10.0, 100.0)
warm_start_space = [True, False]
solver_space     = ['newton-cg', 'sag', 'lbfgs', 'liblinear']


# creating a hyperparameter grid
param_grid = {'C'          : C_space,
              'warm_start' : warm_start_space,
              'solver'     : solver_space}


# INSTANTIATING the model object without hyperparameters
lr_tuned = LogisticRegression(random_state = 219,
                              max_iter     = 5000)


# GridSearchCV object
lr_tuned_cv = RandomizedSearchCV(estimator           = lr_tuned,   # the model object
                                 param_distributions = param_grid, # parameters to tune
                                 cv                  = 3,          # how many folds in cross-validation
                                 n_iter              = 8,        # number of combinations of hyperparameters to try
                                 random_state        = 219,        # starting point for random sequence
                                 scoring = make_scorer(
                                           roc_auc_score,
                                           needs_threshold = False)) # scoring criteria (AUC)


# FITTING to the FULL DATASET (due to cross-validation)
lr_tuned_cv.fit(chef_data_full, chef_target_full)


# PREDICT step is not needed


# printing the optimal parameters and best score
print("Tuned Parameters  :", lr_tuned_cv.best_params_)
print("Tuned CV AUC      :", lr_tuned_cv.best_score_.round(4))

<ipython-input-30-ecd2318cd68f>:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  C_space          = pd.np.arange(1.0, 10.0, 100.0)
/Users/arlinegarin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/arlinegarin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/arlinegarin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/arlinegarin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear

Tuned Parameters  : {'warm_start': True, 'solver': 'liblinear', 'C': 1.0}
Tuned CV AUC      : 0.6792


In [31]:
# building a model based on hyperparameter tuning results

# INSTANTIATING a logistic regression model with tuned values
lr_tuned = lr_tuned_cv.best_estimator_


# FIT step is not needed


# PREDICTING based on the testing set
lr_tuned_pred = lr_tuned.predict(X_test)


# SCORING the results
print('LR Tuned Training ACCURACY:', lr_tuned.score(X_train, y_train).round(4))
print('LR Tuned Testing  ACCURACY:', lr_tuned.score(X_test, y_test).round(4))
print('LR Tuned AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = lr_tuned_pred).round(4))


# saving scoring data for future use
lr_tuned_train_score = lr_tuned.score(X_train, y_train).round(4) # accuracy
lr_tuned_test_score  = lr_tuned.score(X_test, y_test).round(4)   # accuracy


# saving the AUC score
lr_tuned_auc         = roc_auc_score(y_true  = y_test,
                                     y_score = lr_tuned_pred).round(4) # auc

LR Tuned Training ACCURACY: 0.767
LR Tuned Testing  ACCURACY: 0.7762
LR Tuned AUC Score        : 0.7015


In [32]:
# unpacking the confusion matrix
lr_tuned_tn, \
lr_tuned_fp, \
lr_tuned_fn, \
lr_tuned_tp = confusion_matrix(y_true = y_test, y_pred = lr_tuned_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {lr_tuned_tn}
False Positives: {lr_tuned_fp}
False Negatives: {lr_tuned_fn}
True Positives : {lr_tuned_tp}
""")


True Negatives : 77
False Positives: 79
False Negatives: 30
True Positives : 301



In [33]:
# declaring model performance objects
lr_train_acc = lr_tuned.score(X_train, y_train).round(4)
lr_test_acc  = lr_tuned.score(X_test, y_test).round(4)
lr_auc       = roc_auc_score(y_true  = y_test,
                             y_score = lr_tuned_pred).round(4)


# checking the results

# creating a dictionary for model results
model_performance = {
    
    'Model Name'    : ['Logistic', 'Tuned LR'],
           
    'AUC Score' : [logreg_auc_score, lr_auc],
    
    'Training Accuracy' : [logreg_train_score, lr_train_acc],
           
    'Testing Accuracy'  : [logreg_test_score, lr_test_acc],

    'Confusion Matrix'  : [(logreg_tn, logreg_fp, logreg_fn, logreg_tp),
                           (lr_tuned_tn, lr_tuned_fp, lr_tuned_fn, lr_tuned_tp)]}


# converting model_performance into a DataFrame
model_performance = pd.DataFrame(model_performance)

## Classification Trees

### Full Tree

In [34]:
# INSTANTIATING a classification tree object
full_tree = DecisionTreeClassifier()


# FITTING the training data
full_tree_fit = full_tree.fit(X_train, y_train)


# PREDICTING on new data
full_tree_pred = full_tree_fit.predict(X_test)


# SCORING the model
print('Full Tree Training Score:', full_tree_fit.score(X_train,
                                                    y_train).round(4))

print('Full Tree Testing Score :', full_tree_fit.score(X_test,
                                                    y_test).round(4))

print('Full Tree AUC Score:', roc_auc_score(y_true  = y_test,
                                            y_score = full_tree_pred).round(4))


# saving scoring data for future use
full_tree_train_score = full_tree_fit.score(X_train, y_train).round(4) # accuracy
full_tree_test_score  = full_tree_fit.score(X_test, y_test).round(4)   # accuracy


# saving AUC
full_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                      y_score = full_tree_pred).round(4) # auc

Full Tree Training Score: 1.0
Full Tree Testing Score : 0.6263
Full Tree AUC Score: 0.5861


In [35]:
# unpacking the confusion matrix
full_tree_tn, \
full_tree_fp, \
full_tree_fn, \
full_tree_tp = confusion_matrix(y_true = y_test, y_pred = full_tree_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {full_tree_tn}
False Positives: {full_tree_fp}
False Negatives: {full_tree_fn}
True Positives : {full_tree_tp}
""")


True Negatives : 74
False Positives: 82
False Negatives: 100
True Positives : 231



In [36]:
# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'        : 'Full Tree',
                           'Training Accuracy' : full_tree_train_score,
                           'Testing Accuracy'  : full_tree_test_score,
                           'AUC Score'         : full_tree_auc_score,
                           'Confusion Matrix'  : (full_tree_tn,
                                                    full_tree_fp,
                                                    full_tree_fn,
                                                    full_tree_tp)},
                           ignore_index = True)

### Tree Pruned

In [37]:
# INSTANTIATING a classification tree object
tree_pruned = DecisionTreeClassifier(max_depth = 3,
                    min_samples_leaf = 25,
                    random_state = 219)


# FITTING the training data
tree_pruned_fit = tree_pruned.fit(X_train, y_train)


# PREDICTING on new data
tree_pred = tree_pruned_fit.predict(X_test)


# SCORING the model
print('Training ACCURACY:', tree_pruned_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', tree_pruned_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_pred).round(4))


# saving scoring data for future use
pruned_tree_train_score = tree_pruned_fit.score(X_train, y_train).round(4) # accuracy
pruned_tree_test_score  = tree_pruned_fit.score(X_test, y_test).round(4) # accuracy


# saving auc score
pruned_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                        y_score = tree_pred).round(4) # auc

Training ACCURACY: 0.7409
Testing  ACCURACY: 0.7762
AUC Score        : 0.732


In [38]:
# unpacking the confusion matrix
pruned_tree_tn, \
pruned_tree_fp, \
pruned_tree_fn, \
pruned_tree_tp = confusion_matrix(y_true = y_test, y_pred = tree_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {pruned_tree_tn}
False Positives: {pruned_tree_fp}
False Negatives: {pruned_tree_fn}
True Positives : {pruned_tree_tp}
""")


True Negatives : 95
False Positives: 61
False Negatives: 48
True Positives : 283



In [39]:
# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'        : 'Pruned Tree',
                           'Training Accuracy' : pruned_tree_train_score,
                           'Testing Accuracy'  : pruned_tree_test_score,
                           'AUC Score'         : pruned_tree_auc_score,
                           'Confusion Matrix'  : (pruned_tree_tn,
                                                    pruned_tree_fp,
                                                    pruned_tree_fn,
                                                    pruned_tree_tp)},
                           ignore_index = True)

model_performance

,Model Name,AUC Score,Training Accuracy,Testing Accuracy,Confusion Matrix
0,Logistic,0.6858,0.7608,0.7618,"(74, 82, 34, 297)"
1,Tuned LR,0.7015,0.7670,0.7762,"(77, 79, 30, 301)"
2,Full Tree,0.5861,1.0000,0.6263,"(74, 82, 100, 231)"
3,Pruned Tree,0.7320,0.7409,0.7762,"(95, 61, 48, 283)"


### Hyperparameter Tuning On Classification Trees

In [40]:
# declaring a hyperparameter space
criterion_space = ['gini', 'entropy']
splitter_space  = ['best', 'random']
depth_space     = pd.np.arange(1, 8, 1)
leaf_space      = pd.np.arange(1, 100, 1)


# creating a hyperparameter grid
param_grid = {'criterion'        : criterion_space,
              'splitter'         : splitter_space,
              'max_depth'        : depth_space,
              'min_samples_leaf' : leaf_space}


# INSTANTIATING the model object without hyperparameters
tuned_tree = DecisionTreeClassifier(random_state = 219)


# RandomizedSearchCV object
tuned_tree_cv = RandomizedSearchCV(estimator             = tuned_tree,
                                   param_distributions   = param_grid,
                                   cv                    = 8,
                                   n_iter                = 250,
                                   random_state          = 219,
                                   scoring = make_scorer(roc_auc_score,
                                             needs_threshold = False))


# FITTING to the FULL DATASET (due to cross-validation)
tuned_tree_cv.fit(chef_data_full, chef_target_full)


# PREDICT step is not needed


# printing the optimal parameters and best score
print("Tuned Parameters  :", tuned_tree_cv.best_params_)
print("Tuned Training AUC:", tuned_tree_cv.best_score_.round(4))

<ipython-input-40-13e780f9600f>:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  depth_space     = pd.np.arange(1, 8, 1)
<ipython-input-40-13e780f9600f>:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  leaf_space      = pd.np.arange(1, 100, 1)


Tuned Parameters  : {'splitter': 'best', 'min_samples_leaf': 8, 'max_depth': 3, 'criterion': 'entropy'}
Tuned Training AUC: 0.704


In [41]:
# building a model based on hyperparameter tuning results

# INSTANTIATING a logistic regression model with tuned values
tree_tuned = tuned_tree_cv.best_estimator_


# FIT step is not needed


# PREDICTING based on the testing set
tree_tuned_pred = tree_tuned.predict(X_test)


# SCORING the results
print('Training ACCURACY:', tree_tuned.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', tree_tuned.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_tuned_pred).round(4))


# saving scoring data for future use
tree_tuned_train_score = tree_tuned.score(X_train, y_train).round(4) # accuracy
tree_tuned_test_score  = tree_tuned.score(X_test, y_test).round(4)   # accuracy


# saving the AUC score
tree_tuned_auc         = roc_auc_score(y_true  = y_test,
                                     y_score = tree_tuned_pred).round(4) # auc

Training ACCURACY: 0.7409
Testing  ACCURACY: 0.7762
AUC Score        : 0.732


In [42]:
# unpacking the confusion matrix
tuned_tree_tn, \
tuned_tree_fp, \
tuned_tree_fn, \
tuned_tree_tp = confusion_matrix(y_true = y_test, y_pred = tree_tuned_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {tuned_tree_tn}
False Positives: {tuned_tree_fp}
False Negatives: {tuned_tree_fn}
True Positives : {tuned_tree_tp}
""")


True Negatives : 95
False Positives: 61
False Negatives: 48
True Positives : 283



In [43]:
# declaring model performance objects
tree_train_acc = tree_tuned.score(X_train, y_train).round(4)
tree_test_acc  = tree_tuned.score(X_test, y_test).round(4)
tree_auc       = roc_auc_score(y_true  = y_test,
                              y_score = tree_tuned_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'        : 'Tuned Tree',
                           'Training Accuracy' : tree_train_acc,
                           'Testing Accuracy'  : tree_test_acc,
                           'AUC Score'         : tree_auc,
                           'Confusion Matrix'  : (tuned_tree_tn,
                                                  tuned_tree_fp,
                                                  tuned_tree_fn,
                                                  tuned_tree_tp)},
                           ignore_index = True)

## Random Forest

The following cells for the Random Forest Model are not being run as the scores are not relevant enough for this project.

In [44]:
# INSTANTIATING a random forest model with default values
#rf_default = RandomForestClassifier(n_estimators     = 100,
#                                    criterion        = 'gini',
#                                    max_depth        = 4,
#                                    min_samples_leaf = 1,
#                                    bootstrap        = True,
#                                    warm_start       = False,
#                                    random_state     = 219)

In [45]:
# FITTING the training data
#rf_default_fit = rf_default.fit(X_train, y_train)


# PREDICTING based on the testing set
#rf_default_fit_pred = rf_default_fit.predict(X_test)


# SCORING the results
#print('Training Random Forest:', rf_default_fit.score(X_train, y_train).round(4))
#print('Testing  Random Forest:', rf_default_fit.score(X_test, y_test).round(4))


# saving AUC score
#print('AUC Score        :', roc_auc_score(y_true  = y_test,
#                                          y_score = rf_default_fit_pred).round(4))

In [46]:
# unpacking the confusion matrix
#rf_tn, \
#rf_fp, \
#rf_fn, \
#rf_tp = confusion_matrix(y_true = y_test, y_pred = rf_default_fit_pred).ravel()


# printing each result one-by-one
#print(f"""
#True Negatives : {rf_tn}
#False Positives: {rf_fp}
#False Negatives: {rf_fn}
#True Positives : {rf_tp}
#""")

In [47]:
# declaring model performance objects
#rf_train_acc = rf_default_fit.score(X_train, y_train).round(4)
#rf_test_acc  = rf_default_fit.score(X_test, y_test).round(4)
#rf_auc       = roc_auc_score(y_true  = y_test,
#                             y_score = rf_default_fit_pred).round(4)


# appending to model_performance
#model_performance = model_performance.append(
#                          {'Model Name'         : 'Random Forest (Full)',
#                           'Training Accuracy'  : rf_train_acc,
#                           'Testing Accuracy'   : rf_test_acc,
#                           'AUC Score'          : rf_auc,
#                           'Confusion Matrix'   : (rf_tn,
#                                                   rf_fp,
#                                                   rf_fn,
#                                                   rf_tp)},
#                          ignore_index = True)


# checking the results
#model_performance

### Hyperparameter Tuning

In [48]:
# FITTING the training data
#rf_default_fit = rf_default.fit(X_train, y_train)


# PREDICTING based on the testing set
#rf_default_fit_pred = rf_default_fit.predict(X_test)


# declaring a hyperparameter space
#estimator_space  = pd.np.arange(100, 500, 250)
#leaf_space       = pd.np.arange(1, 31, 10)
#criterion_space  = ['gini', 'entropy']


# creating a hyperparameter grid
#param_grid = {'n_estimators'     : estimator_space,
#              'min_samples_leaf' : leaf_space,
#              'criterion'        : criterion_space}


# INSTANTIATING the model object without hyperparameters
#forest_grid = RandomForestClassifier(random_state = 219)


# GridSearchCV object
#forest_cv = RandomizedSearchCV(estimator           = forest_grid,
#                               param_distributions = param_grid,
#                               cv         = 6,
#                               n_iter     = 12,
#                               scoring    = make_scorer(roc_auc_score,
#                                            needs_threshold = False))


# FITTING to the FULL DATASET (due to cross-validation)
#forest_cv.fit(chef_data_full, chef_target_full)


# PREDICT step is not needed


# printing the optimal parameters and best score
#print("Tuned Parameters  :", forest_cv.best_params_)
#print("Tuned Training AUC:", forest_cv.best_score_.round(4))

In [49]:
# building a model based on hyperparameter tuning results

# INSTANTIATING with best_estimator
#forest_tuned = forest_cv.best_estimator_


# FIT step not needed


# PREDICTING based on the testing set
#forest_tuned_pred = forest_tuned.predict(X_test)


# SCORING the results
#print('Training Forest Tuned:', forest_tuned.score(X_train, y_train).round(4))
#print('Testing Forest Tuned :', forest_tuned.score(X_test, y_test).round(4))
#print('AUC Score Forest Tuned        :', roc_auc_score(y_true  = y_test,
#                                                       y_score = forest_tuned_pred).round(4))


# saving scoring data for future use
#forest_tuned_train_score = forest_tuned.score(X_train, y_train).round(4) # accuracy
#forest_tuned_test_score  = forest_tuned.score(X_test, y_test).round(4)   # accuracy


# saving the AUC score
#forest_tuned_auc = roc_auc_score(y_true  = y_test,
#                                 y_score = forest_tuned_pred).round(4) # auc

In [50]:
# unpacking the confusion matrix
#tuned_rf_tn, \
#tuned_rf_fp, \
#tuned_rf_fn, \
#tuned_rf_tp = confusion_matrix(y_true = y_test, y_pred = forest_tuned_pred).ravel()


# printing each result one-by-one
#print(f"""
#True Negatives : {tuned_rf_tn}
#False Positives: {tuned_rf_fp}
#False Negatives: {tuned_rf_fn}
#True Positives : {tuned_rf_tp}
#""")

In [51]:
# declaring model performance objects
#tuned_rf_train_acc = forest_tuned.score(X_train, y_train).round(4)
#tuned_rf_test_acc  = forest_tuned.score(X_test, y_test).round(4)
#tuned_rf_auc       = roc_auc_score(y_true  = y_test,
#                                   y_score = forest_tuned_pred).round(4)


# appending to model_performance
#model_performance = model_performance.append(
#                          {'Model Name'         : 'Tuned Random Forest (Full)',
#                           'Training Accuracy'  : tuned_rf_train_acc,
#                           'Testing Accuracy'   : tuned_rf_test_acc,
#                           'AUC Score'          : tuned_rf_auc,
#                           'Confusion Matrix'   : (tuned_rf_tn,
#                                                   tuned_rf_fp,
#                                                   tuned_rf_fn,
#                                                   tuned_rf_tp)},
#                          ignore_index = True)


# checking the results
#model_performance

## Gradient Boosted Machines

In [52]:
# INSTANTIATING a classification 
g_boost = GradientBoostingClassifier(loss = 'deviance',
                                     criterion = 'friedman_mse',
                                     learning_rate =  0.1,
                                     n_estimators = 100,
                                     max_depth = 4,
                                     random_state  = 219)

# FITTING the training data
g_boost_fit = g_boost.fit(X_train, y_train)

# PREDICTING on test data
g_boost_pred = g_boost_fit.predict(X_test)

# SCORING the model
print('Training ACCURACY:', g_boost_fit.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', g_boost_fit.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = g_boost_pred).round(4))


Training ACCURACY: 0.904
Testing  ACCURACY: 0.7454
AUC Score        : 0.6619


In [53]:
# unpacking the confusion matrix
gbm_default_tn, \
gbm_default_fp, \
gbm_default_fn, \
gbm_default_tp = confusion_matrix(y_true = y_test, y_pred = g_boost_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_default_tn}
False Positives: {gbm_default_fp}
False Negatives: {gbm_default_fn}
True Positives : {gbm_default_tp}
""")


True Negatives : 67
False Positives: 89
False Negatives: 35
True Positives : 296



In [54]:
# SCORING the model
gbm_train_acc = g_boost_fit.score(X_train, y_train).round(4)
gbm_test_acc  = g_boost_fit.score(X_test, y_test).round(4)
gbm_auc       = roc_auc_score(y_true  = y_test,
                              y_score = g_boost_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'       : 'GBM - Full',
                          'Training Accuracy' : gbm_train_acc,
                          'Testing Accuracy'  : gbm_test_acc,
                          'AUC Score'         : gbm_auc,
                          'Confusion Matrix'  : (gbm_default_tn,
                                                 gbm_default_fp,
                                                 gbm_default_fn,
                                                 gbm_default_tp)},
                          ignore_index = True)

### Hyperparameter Tuning

In [55]:
# declaring a hyperparameter space
learn_space        = pd.np.arange(0.1, 2.0, 0.2) # play with this one
estimator_space    = pd.np.arange(50, 100, 25) # play with this one


# creating a hyperparameter grid
param_grid = {'learning_rate' : learn_space,
              'n_estimators'  : estimator_space}


# INSTANTIATING the model object without hyperparameters
full_gbm_grid = GradientBoostingClassifier(max_depth = 4,
                                           random_state  = 219)


# GridSearchCV object
full_gbm_cv = RandomizedSearchCV(estimator = full_gbm_grid,
                                 param_distributions = param_grid,
                                 cv                  = 3,
                                 n_iter              = 20,
                                 random_state        = 219,
                                 scoring             = make_scorer(roc_auc_score,
                                                       needs_threshold = False))


# FITTING to the FULL DATASET (due to cross-validation)
full_gbm_cv.fit(chef_data_full, chef_target_full)


# PREDICT step is not needed


# printing the optimal parameters and best score
print("Tuned Parameters  :", full_gbm_cv.best_params_)
print("Tuned Training AUC:", full_gbm_cv.best_score_.round(4))

<ipython-input-55-7cb13a22c9d9>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  learn_space        = pd.np.arange(0.1, 2.0, 0.2) # play with this one
<ipython-input-55-7cb13a22c9d9>:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  estimator_space    = pd.np.arange(50, 100, 25) # play with this one


Tuned Parameters  : {'n_estimators': 50, 'learning_rate': 0.30000000000000004}
Tuned Training AUC: 0.6412


In [56]:
# INSTANTIATING with best_estimator
gbm_tuned = full_gbm_cv.best_estimator_


# FIT step not needed


# PREDICTING based on the testing set
gbm_tuned_pred = gbm_tuned.predict(X_test)


# SCORING the results
print('Training ACCURACY:', gbm_tuned.score(X_train, y_train).round(4))
print('Testing  ACCURACY:', gbm_tuned.score(X_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = gbm_tuned_pred).round(4))

Training ACCURACY: 0.9191
Testing  ACCURACY: 0.9322
AUC Score        : 0.9061


In [57]:
# unpacking the confusion matrix
gbm_tuned_tn, \
gbm_tuned_fp, \
gbm_tuned_fn, \
gbm_tuned_tp = confusion_matrix(y_true = y_test, y_pred = gbm_tuned_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_tuned_tn}
False Positives: {gbm_tuned_fp}
False Negatives: {gbm_tuned_fn}
True Positives : {gbm_tuned_tp}
""")


True Negatives : 130
False Positives: 26
False Negatives: 7
True Positives : 324



In [58]:
# declaring model performance objects
gbm_tuned_train_acc = gbm_tuned.score(X_train, y_train).round(4)
gbm_tuned_test_acc  = gbm_tuned.score(X_test, y_test).round(4)
gbm_tuned_auc       = roc_auc_score(y_true  = y_test,
                              y_score = gbm_tuned_pred).round(4)


# appending to model_performance
model_performance = model_performance.append(
                          {'Model Name'        : '**Tuned GBM (final model)',
                          'Training Accuracy'  : gbm_tuned_train_acc,
                          'Testing Accuracy'   : gbm_tuned_test_acc,
                          'AUC Score'          : gbm_tuned_auc,
                          'Confusion Matrix'   : (gbm_tuned_tn,
                                                  gbm_tuned_fp,
                                                  gbm_tuned_fn,
                                                  gbm_tuned_tp)},
                          ignore_index = True)

# sending model results to Excel
model_performance.to_excel('./classification_model_performance.xlsx',
                           index = False)

## Final Output

In [59]:
model_performance.sort_values(by = 'AUC Score',
                              ascending = False)

,Model Name,AUC Score,Training Accuracy,Testing Accuracy,Confusion Matrix
6,**Tuned GBM (final model),0.9061,0.9191,0.9322,"(130, 26, 7, 324)"
3,Pruned Tree,0.7320,0.7409,0.7762,"(95, 61, 48, 283)"
4,Tuned Tree,0.7320,0.7409,0.7762,"(95, 61, 48, 283)"
1,Tuned LR,0.7015,0.7670,0.7762,"(77, 79, 30, 301)"
0,Logistic,0.6858,0.7608,0.7618,"(74, 82, 34, 297)"
5,GBM - Full,0.6619,0.9040,0.7454,"(67, 89, 35, 296)"
2,Full Tree,0.5861,1.0000,0.6263,"(74, 82, 100, 231)"


In [60]:
print(f"""

MODEL OUTPUTS
-------------

Model Name              AUC Score         Training Accuracy      Testing Accuracy      Confusion Matrix
----------              ---------         -----------------      ----------------      ----------------
Logistic                {logreg_auc_score}            {logreg_train_score}                 {logreg_test_score}                {(logreg_tn, logreg_fp, logreg_fn, logreg_tp)}
Tuned LR                {lr_auc}            {lr_train_acc}                  {lr_test_acc}                {(lr_tuned_tn, lr_tuned_fp, lr_tuned_fn, lr_tuned_tp)}
Full Tree               {full_tree_auc_score}            {full_tree_train_score}                    {full_tree_test_score}                {(full_tree_tn, full_tree_fp, full_tree_fn, full_tree_tp)}
Pruned Tree             {pruned_tree_auc_score}             {pruned_tree_train_score}                 {pruned_tree_test_score}                {(pruned_tree_tn, pruned_tree_fp, pruned_tree_fn, pruned_tree_tp)}
Tuned Tree              {tree_auc}             {tree_train_acc}                 {tree_test_acc}                {(tuned_tree_tn, tuned_tree_fp, tuned_tree_fn, tuned_tree_tp)}
GBM - Full              {gbm_auc}            {gbm_train_acc}                  {gbm_test_acc}                {(gbm_default_tn, gbm_default_fp, gbm_default_fn, gbm_default_tp)}
*Tuned GBM (final)      {gbm_tuned_auc}            {gbm_tuned_train_acc}                 {gbm_tuned_test_acc}                {(gbm_tuned_tn, gbm_tuned_fp, gbm_tuned_fn, gbm_tuned_tp)}

** The best model is the Tuned GBM Model with the full dataset and a AUC score of 0.9061. 
The train test-gap is 0.0131.
""")



MODEL OUTPUTS
-------------

Model Name              AUC Score         Training Accuracy      Testing Accuracy      Confusion Matrix
----------              ---------         -----------------      ----------------      ----------------
Logistic                0.6858            0.7608                 0.7618                (74, 82, 34, 297)
Tuned LR                0.7015            0.767                  0.7762                (77, 79, 30, 301)
Full Tree               0.5861            1.0                    0.6263                (74, 82, 100, 231)
Pruned Tree             0.732             0.7409                 0.7762                (95, 61, 48, 283)
Tuned Tree              0.732             0.7409                 0.7762                (95, 61, 48, 283)
GBM - Full              0.6619            0.904                  0.7454                (67, 89, 35, 296)
*Tuned GBM (final)      0.9061            0.9191                 0.9322                (130, 26, 7, 324)

** The best model is the